In [1]:
# load module
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import wntr
import wntr_utils
from sklearn.preprocessing import MinMaxScaler
from random import shuffle
import csv
import random

In [18]:
week_index = 1
total_demands = pd.read_csv('three_year_demands_ctown.csv', index_col=0)
demand_starting_points = pd.read_csv('starting_demand_points.csv', index_col=0)
initial_tank_levels = pd.read_csv('tank_initial_conditions.csv', index_col=0)
network_delays = pd.read_csv('network_links_delay.csv', index_col=0)
network_losses = pd.read_csv('network_loss_small.csv', index_col=0)

In [3]:
# load EPANET model
inp_file = 'ctown_map.inp'
wn = wntr.network.WaterNetworkModel(inp_file)

C:\ProgramData\Anaconda3\lib\site-packages\wntr\epanet\io.py:2085: UserWarning:

Not all curves were used in "ctown_map.inp"; added with type None, units conversion left to user



In [4]:
week_index = 1-1
week_start = demand_starting_points.iloc[week_index][0]

In [5]:
week_demands = total_demands.loc[week_start:week_start+167, :]

In [6]:
week_demands

,DMA1_pat,DMA2_pat,DMA3_pat,DMA4_pat,DMA5_pat
15464,0.819188,0.781149,0.592089,0.755898,0.728942
15465,0.632566,0.636582,0.659333,0.624151,0.715041
15466,0.634120,0.608031,0.594177,0.601135,0.690219
15467,0.639076,0.592978,0.626087,0.619306,0.560457
15468,0.538156,0.607862,0.567203,0.564579,0.551973
...,...,...,...,...,...
15627,0.363170,0.369905,0.334993,0.367948,0.395216
15628,0.427679,0.461839,0.404623,0.368017,0.424950
15629,0.537413,0.581188,0.555527,0.535370,0.508833
15630,0.639536,0.688143,0.645961,0.637576,0.675752


In [7]:
week_demands['DMA1_pat'].values.tolist()

[0.8191877364722848,
 0.6325660034591887,
 0.6341204630791828,
 0.6390756987981645,
 0.5381562327145257,
 0.4910590516039218,
 0.5652876139743118,
 0.5549701398393481,
 0.5361089930481976,
 0.6023396116491657,
 0.5672703192479366,
 0.5117496889231092,
 0.5282539755727508,
 0.5191107970163803,
 0.3842987405519968,
 0.4818619831798832,
 0.425599906924189,
 0.3503165305432185,
 0.3273750762001895,
 0.3941673609879941,
 0.35988279962100883,
 0.6225811926131085,
 0.6907197696603687,
 0.6430863297564113,
 0.708538693185966,
 0.6756213857858538,
 0.6432573843487477,
 0.6081110570996863,
 0.5655013330539101,
 0.5526817708411236,
 0.5042709652241796,
 0.5383581748570829,
 0.5580269302138512,
 0.5471720967824447,
 0.5593806974648693,
 0.5872870040716641,
 0.5389318688344256,
 0.4661129878165974,
 0.4375668385627599,
 0.4793323203154146,
 0.4863375062107183,
 0.3347363571391753,
 0.3452538481014608,
 0.3374680935655828,
 0.3914570155074681,
 0.4943786353205677,
 0.7025725955758622,
 0.69289358126

In [9]:
# Replace the demand patterns with the random patterns for this week
names, patterns =[],[]
for name, pat in wn.patterns():  
    pat.multipliers = week_demands[name].values.tolist()

In [10]:
for name, pat in wn.patterns():  
    print(name)
    print(pat.multipliers)

DMA1_pat
[0.819 0.633 0.634 0.639 0.538 0.491 0.565 0.555 0.536 0.602 0.567 0.512 0.528 0.519 0.384 0.482 0.426 0.35  0.327 0.394 0.36  0.623 0.691 0.643 0.709 0.676 0.643 0.608 0.566 0.553 0.504 0.538 0.558 0.547 0.559 0.587 0.539 0.466 0.438 0.479 0.486 0.335 0.345 0.337 0.391 0.494 0.703 0.693 0.693
 0.738 0.565 0.641 0.599 0.533 0.534 0.536 0.538 0.547 0.566 0.599 0.537 0.48  0.468 0.539 0.42  0.341 0.4   0.367 0.405 0.481 0.625 0.781 0.731 0.7   0.613 0.596 0.633 0.596 0.49  0.494 0.566 0.543 0.514 0.585 0.492 0.502 0.457 0.624 0.475 0.325 0.346 0.339 0.42  0.522 0.643 0.772 0.727 0.723
 0.644 0.613 0.575 0.583 0.546 0.556 0.6   0.63  0.553 0.57  0.522 0.465 0.443 0.606 0.473 0.354 0.329 0.366 0.427 0.504 0.702 0.727 0.754 0.723 0.612 0.594 0.554 0.538 0.587 0.508 0.606 0.627 0.542 0.518 0.586 0.427 0.397 0.482 0.438 0.365 0.352 0.348 0.399 0.473 0.661 0.656 0.727 0.686 0.671
 0.635 0.514 0.519 0.517 0.542 0.629 0.601 0.556 0.587 0.477 0.487 0.411 0.519 0.567 0.368 0.347 0.363 0.4

In [11]:
wn.options.time.duration=len(week_demands.values)*3600

In [17]:
initial_tank_levels.iloc[week_index]

T1    5.319035
T2    1.294841
T3    4.277594
T4    3.547004
T5    3.844998
T6    4.195264
T7    1.271854
Name: 1, dtype: float64

In [15]:
for i in range(1,8):    
    wn.get_node('T' + str(i)).init_level = float(initial_tank_levels.iloc[week_index]['T'+ str(i)])

In [16]:
for i in range(1,8):    
    print(wn.get_node('T' + str(i)).init_level)

5.319035490181273
1.294840707464302
4.277593948430703
3.5470040060252948
3.8449984876444017
4.195263786801192
1.271853618792292


In [12]:
# Simulate hydraulics
sim = wntr.sim.WNTRSimulator(wn,mode='PDD')

In [13]:
results_new = sim.run_sim()

In [15]:
pressure_results = results_new.node['pressure']
pressure_results.to_csv('test_ctown_week_1_wntr.csv')

In [28]:
network_delays.iloc[0]['r1']

21.208797452631494

In [27]:
network_losses.iloc[0]['r1']

0.2448607788241941